In [1]:
!pip install coremltools==4.0b1 --quiet

     |████████████████████████████████| 604kB 4.5MB/s 


In [2]:
import shutil
import coremltools as ct

from fastai import *
from fastai.vision import *
from fastai.metrics import accuracy

import warnings
warnings.filterwarnings('ignore')

## Import Dataset from Kaggle

In [3]:
!wget -x --load-cookies cookies.txt 'https://www.kaggle.com/ayuraj/american-sign-language-dataset/download' -O data.zip
!unzip -q data.zip
shutil.rmtree('/content/asl/asl')

--2020-07-27 00:08:25--  https://www.kaggle.com/ayuraj/american-sign-language-dataset/download
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://storage.googleapis.com/kaggle-data-sets/180288%2F405373%2Fbundle%2Farchive.zip?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1596067705&Signature=WTTWMeMr%2BFQGh6u%2FsiVUVhnqdnTABEYm62yVfq4PQ2bNyLyEu2STWsFVFCoUkWsTKUZaDYw%2FnZ4ylyF%2BzIjQ%2FHVBoD5zFxdKsvZAcfosAD3zGCjup8%2BvFqKlZUgVj8ub3DCZDnprZoxpn9nVCBFy37KGUJgTs9bsoZCkaOzO5VU1OukF18zZxAx3ldD%2Fn7LJy%2B2rl%2BxoK4XdHtzUhJWcYMmNeu3KAQuQ6nPgSSnFJmj0VrI8cMKUyIGn47Zgzkok19unIaqYxfFmvaZiroRroRItno1ooMe4T9d4K95SSjjfBqAE0HTd2B%2FCnlMlrowZ%2FQyXDIPynAAYttuYkOElHA%3D%3D [following]
--2020-07-27 00:08:25--  https://storage.googleapis.com/kaggle-data-sets/180288%2F405373%2Fbundle%2Farchive.zip?GoogleAccessId=gcp-kaggle-co

## Train Classifier

In [4]:
batch_size = 64
root = '/content/asl'

In [5]:
files, labels = [], []
for r, d, f in os.walk(root):
    for i in f:
        label = r.split('/')[-1]
        files.append(f'{label}/{i}')
        labels.append(label)
df = pd.DataFrame(list(zip(files, labels)), columns=['name', 'label'])
data = ImageDataBunch.from_df(root, df, size=400, bs=batch_size // 2).normalize(imagenet_stats)

In [6]:
classifier = nn.Sequential(
    nn.AdaptiveAvgPool2d(output_size=1),
    nn.Flatten(),
    nn.BatchNorm1d(1280, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
    nn.Dropout(p=0.25, inplace=False),
    nn.Linear(in_features=1280, out_features=512, bias=True),
    nn.ReLU(inplace=True),
    nn.BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=512, out_features=36, bias=True)
)

learn = cnn_learner(data, models.mobilenet_v2, metrics=accuracy, custom_head=classifier)
learn.fit_one_cycle(5)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/checkpoints/mobilenet_v2-b0353104.pth


epoch,train_loss,valid_loss,accuracy,time
0,0.443431,0.172300,0.938513,03:23
1,0.192343,0.077164,0.976671,03:24
2,0.086537,0.026561,0.991894,03:23
3,0.027171,0.006711,0.997825,03:24
4,0.015318,0.003635,0.999011,03:24


## PyTorch --> CoreML

In [7]:
model = learn.model.cpu().eval()
example_input = torch.rand(1, 3, 400, 400) 
traced_model = torch.jit.trace(model, example_input)

class_labels = data.classes
mlmodel = ct.convert(traced_model,
                     inputs=[ct.ImageType(name='input_1', shape=example_input.shape)], 
                     classifier_config = ct.ClassifierConfig(class_labels))
mlmodel.save('MobileNetV2.mlmodel')

Translating MIL ==> MLModel Ops: 100%|██████████| 817/817 [00:00<00:00, 2058.64 ops/s]
